In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import csv
import glob
import re
import os

## 데이터 불러오기

### 데이터 병합

In [2]:
def merge_file(path):
    '''Concatenate files from path
    Param: path: file path from which data are imported and concatenated
    '''
    file_names = [name for name in glob.glob(path)]
    df_temp = (pd.read_csv(file) for file in file_names)
    return(pd.concat(df_temp, ignore_index=True, axis='index'))

### y_train

In [3]:
path = 'bank-churn-prediction/data/raw/y_train_3/y_Q[34]_3.csv'
y_train = merge_file(path)

In [4]:
y_train.to_csv('bank-churn-prediction/data/preprocess/y_train.csv', index=False)

In [5]:
y_train['cust_no'].duplicated().sum() # id 중복 확인

62397

In [6]:
y_train.duplicated().sum() # 모든 값 중복

40090

=> 22307명의 고객은 3분기 -> 4분기에 따라 이탈 선호가 변화함  
=> 확인해본 결과 같은 분기 내에 cust_no 중복은 없음  
=> 분기에 따라 중복 판단, 같은 분기 안에서는 마지막 기록이 정확하다고 판단해 마지막 기록을 남겨두기로 결정
=> 

### X_train  


In [7]:
path = 'bank-churn-prediction/data/raw/y_train_3/'
y_Q3_3 = pd.read_csv(path + 'y_Q3_3.csv')
y_Q4_3 = pd.read_csv(path + 'y_Q4_3.csv')

In [8]:
# cust_no 열만 남기기
X_train_Q3 = y_Q3_3.drop('label', axis=1).copy()
X_train_Q4 = y_Q4_3.drop('label', axis=1).copy()

1. Customer Assets (aum)  
  
aum_m(Y)  
This set includes customer's asset at the end of month Y.

In [9]:
# Q3
path = 'bank-churn-prediction/data/raw/x_train/aum_m[789].csv'
aum_Q3 = merge_file(path)

In [10]:
# Q4
path = 'bank-churn-prediction/data/raw/x_train/aum_m1[012].csv'
aum_Q4 = merge_file(path)

In [11]:
# Drop duplicated customer IDs except for the last occurrance
## 잔고 --> 마지막 기록만 남기기
aum_Q3 = aum_Q3.drop_duplicates(subset=['cust_no'], keep='last')
aum_Q4 = aum_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [12]:
# # Trim by the customer IDs in `y_train` set, separated by quarter
# aum_Q3 = trim_by_quarter(aum_Q3, True)
# aum_Q4 = trim_by_quarter(aum_Q4, False)

In [13]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(aum_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(aum_Q4, how='left', on='cust_no')

* Customer Behavior (behavior)  
  
behavior_m(Y)  
This set records customers' behaviors in month Y.  
Column B6 and B7 only have data if month == [3, 6, 9, 12]. 

In [14]:
# Q3
path = 'bank-churn-prediction/data/raw/x_train/behavior_m[789].csv'
behavior_Q3 = merge_file(path)

In [15]:
# Q4
path = 'bank-churn-prediction/data/raw/x_train/behavior_m1[012].csv'
behavior_Q4 = merge_file(path)

In [16]:
# Drop duplicated customer IDs except for the last occurrance
## 로그인 횟수, 입금 횟수, 금액 등 -> 마지막만 남겨두기
behavior_Q3 = behavior_Q3.drop_duplicates(subset=['cust_no'], keep='last')
behavior_Q4 = behavior_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [17]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(behavior_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(behavior_Q4, how='left', on='cust_no')

* Important Customer Behavior (big_event) 
  
big_event_Q(Z)  
This set records customers' important behaviors in the season Z.

In [18]:
# Q3
path = 'bank-churn-prediction/data/raw/x_train/big_event_Q3.csv'
big_event_Q3 = merge_file(path)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\concat.py:348: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  objs = list(objs)


In [19]:
# Q4
path = 'bank-churn-prediction/data/raw/x_train/big_event_Q4.csv'
big_event_Q4 = merge_file(path)

In [20]:
## cust_no 중복없음

In [21]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(big_event_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(big_event_Q4, how='left', on='cust_no')

* Customer Deposits (cunkuan)  
  
cunkuan_m(Y)  
This set contains customers' deposits in month Y.

In [22]:
# Q3
path = 'bank-churn-prediction/data/raw/x_train/cunkuan_m[789].csv'
savings_Q3 = merge_file(path)

In [23]:
# Q4
path = 'bank-churn-prediction/data/raw/x_train/cunkuan_m1[012].csv'
savings_Q4 = merge_file(path)

In [24]:
# Drop duplicated customer IDs except for the last occurrance
## 예금 상품 가치, 예금 상품 수 -> 마지막만 남기기
savings_Q3 = savings_Q3.drop_duplicates(subset=['cust_no'], keep='last')
savings_Q4 = savings_Q4.drop_duplicates(subset=['cust_no'], keep='last')

In [25]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(savings_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(savings_Q4, how='left', on='cust_no')

* Valid Customer (cust_avli)  
  
cust_avli_Q(Z)  
This set contains valid customer IDs in the season Z.

In [26]:
# Q3
path = 'bank-churn-prediction/data/raw/x_train/cust_avli_Q3.csv'
cust_avli_Q3 = merge_file(path)

In [27]:
# Q4
path = 'bank-churn-prediction/data/raw/x_train/cust_avli_Q4.csv'
cust_avli_Q4 = merge_file(path)

In [28]:
print(len(cust_avli_Q3.value_counts()) == y_Q3_3.shape[0]) # cust_no 개수 같은지 확인
print(len(cust_avli_Q4.value_counts()) == y_Q4_3.shape[0])

True
True


* Customer Trivias (cust_info)  
  
cust_info_q(Z)  
This set contains customer information in the season Z.

In [29]:
# Q3
path = 'bank-churn-prediction/data/raw/x_train/cust_info_q3.csv'
cust_info_Q3 = merge_file(path)

In [30]:
# Q4
path = 'bank-churn-prediction/data/raw/x_train/cust_info_q4.csv'
cust_info_Q4 = merge_file(path)

In [31]:
## cust_no 중복없음

In [32]:
# Merge to X_train
X_train_Q3 = X_train_Q3.merge(cust_info_Q3, how='left', on='cust_no')
X_train_Q4 = X_train_Q4.merge(cust_info_Q4, how='left', on='cust_no')

X_train ready

In [33]:
print(X_train_Q3.shape)
print(X_train_Q4.shape)

(69126, 56)
(76170, 56)


In [34]:
X_train = X_train_Q3.append(X_train_Q4)
X_train.shape

(145296, 56)

In [35]:
X_train.to_csv('bank-churn-prediction/data/preprocess/X_train.csv', index=False)

## X_test

* Customer Assets (aum)

In [36]:
# Q1
path = 'bank-churn-prediction/data/raw/x_test/aum_m[123].csv'
aum_Q1 = merge_file(path)

In [37]:
aum_Q1['cust_no'].duplicated().sum()

1159826

In [38]:
# Drop duplicated customer IDs except for the last occurrance
aum_Q1 = aum_Q1.drop_duplicates(subset=['cust_no'], keep='last')

* Customer Behavior (behavior)

In [39]:
# Q1
path = 'bank-churn-prediction/data/raw/x_test/behavior_m[123].csv'
behavior_Q1 = merge_file(path)

In [40]:
behavior_Q1.duplicated().sum()

460169

In [41]:
# Drop duplicated customer IDs except for the last occurrance
behavior_Q1 = behavior_Q1.drop_duplicates(subset=['cust_no'], keep='last')

* Important Customer Behavior (big_event)

In [42]:
# Q1
path = 'bank-churn-prediction/data/raw/x_test/big_event_Q1.csv'
big_event_Q1 = merge_file(path)

In [43]:
big_event_Q1['cust_no'].duplicated().sum()

0

* Customer Deposits (cunkuan)

In [44]:
# Q1
path = 'bank-churn-prediction/data/raw/x_test/cunkuan_m[123].csv'
savings_Q1 = merge_file(path)

In [45]:
savings_Q1.duplicated().sum()

247240

In [46]:
# Drop duplicated customer IDs except for the last occurrance
savings_Q1 = savings_Q1.drop_duplicates(subset=['cust_no'], keep='last')

* Valid Customer (cust_avli)

In [47]:
# Q1
path = 'bank-churn-prediction/data/raw/x_test/cust_avli_Q1.csv'
cust_avli_Q1 = merge_file(path)

* Customer Trivias (cust_info)

In [48]:
# Q1
path = 'bank-churn-prediction/data/raw/x_test/cust_info_q1.csv'
cust_info_Q1 = merge_file(path)

In [49]:
cust_info_Q1['cust_no'].duplicated().sum()

0

X_test Ready

In [50]:
X_test = cust_avli_Q1.copy()

In [51]:
X_test = X_test.merge(aum_Q1, how='left', on='cust_no')
X_test = X_test.merge(behavior_Q1, how='left', on='cust_no')
X_test = X_test.merge(big_event_Q1, how='left', on='cust_no')
X_test = X_test.merge(savings_Q1, how='left', on='cust_no')
X_test = X_test.merge(cust_info_Q1, how='left', on='cust_no')
X_test.shape

(76722, 56)

In [52]:
X_test.to_csv('bank-churn-prediction/data/preprocess/X_test.csv', index=False)

## 전처리  
  
-> 고려해야할 것:  
어떤 열을 선택할 지, 결측치 어떻게 처리할 지

In [121]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

In [118]:
X = pd.read_csv('bank-churn-prediction/data/preprocess/X_train.csv')
y = pd.read_csv('bank-churn-prediction/data/preprocess/y_train.csv')
X_true = pd.read_csv('bank-churn-prediction/data/preprocess/X_test.csv')

### y (Label for Validation)

In [119]:
y['label'].value_counts() / len(y) * 100

 1    63.882006
 0    20.810621
-1    15.307373
Name: label, dtype: float64

In [122]:
lb = LabelBinarizer()
lb.fit(y['label'])

LabelBinarizer()

In [123]:
lb.classes_

array([-1,  0,  1], dtype=int64)

In [124]:
y_label = lb.transform(y['label'])

### X (Feature for Training)

1. 열별 결측치 비율 확인  
결측치 50% 이상이면 의미 없지 않을까...?  

**E4**(58.15%) : first online banking login date  
**E7**(98%) : first time deposit date (최초 입금 일)   
**E8**(87.67%) : first loan date (최초 대출 일)  
**E9**(99.95%) : first overdue date (최초 연체 날짜)  
**E11**(100%) : first bank-securities transfer date (최초 은행 - 증권 이체 날짜)  
**E12**(83.5%) : first transfer at counter date (최초 카운터에서 이체 날짜)  
**E13**(87.75%) : first transfer via online banking date (온라인 뱅킹을 이용한 최초 이체 날짜)  
**E14**(61.95%) : first transfer via mobile banking date (모바일 뱅킹을 이용한 최초 이체 날짜)   
**I9**(100%) : contribution (기부금)  
**I10**(88.43%) : education level (교육 수준)  
**I13**(98.49%) : marriage description
**I14**(89.23%) : occupation description

In [127]:
X.isnull().sum()/len(X) * 100

cust_no      0.000000
X1           0.000000
X2           0.000000
X3           0.000000
X4           0.000000
X5           0.000000
X6           0.000000
X7           0.000000
X8           0.000000
B1           0.000000
B2           0.000000
B3           0.000000
B4           0.000000
B5           0.000000
B6           6.110285
B7           0.000000
E1           0.000000
E2           4.384154
E3           4.384154
E4          58.145441
E5          37.942545
E6           5.188030
E7          98.008204
E8          87.669998
E9          99.952511
E10          0.561612
E11        100.000000
E12         83.501266
E13         87.753276
E14         61.949400
E15          0.000000
E16         47.165786
E17          0.000000
E18         42.772685
C1           0.004818
C2           0.004818
I1           0.044048
I2           0.000000
I3           0.000000
I4           0.000000
I5           7.986455
I6           0.000000
I7           0.000000
I8           0.000000
I9         100.000000
I10       

In [131]:
# Drop columns with large portion of missing values
col_to_drop = ['E4', 'E7', 'E8', 'E9', 'E11', 'E12', 'E13', 'E14', 'I9', 'I10', 'I13', 'I14']
X = X.drop(col_to_drop, axis=1)

In [133]:
X.isnull().sum()/len(X) * 100

cust_no     0.000000
X1          0.000000
X2          0.000000
X3          0.000000
X4          0.000000
X5          0.000000
X6          0.000000
X7          0.000000
X8          0.000000
B1          0.000000
B2          0.000000
B3          0.000000
B4          0.000000
B5          0.000000
B6          6.110285
B7          0.000000
E1          0.000000
E2          4.384154
E3          4.384154
E5         37.942545
E6          5.188030
E10         0.561612
E15         0.000000
E16        47.165786
E17         0.000000
E18        42.772685
C1          0.004818
C2          0.004818
I1          0.044048
I2          0.000000
I3          0.000000
I4          0.000000
I5          7.986455
I6          0.000000
I7          0.000000
I8          0.000000
I11         0.000000
I12         0.000000
I15         0.000000
I16         0.000000
I17         0.000000
I18         0.000000
I19         0.000000
I20         0.000000
dtype: float64

**결측치 존재: ['B6', 'E2', 'E3', 'E5', 'E6', 'E10', 'E16', 'E18', 'C1', 'C2', 'I1', 'I5']**

#### customer's asset at the end of month Y.

- X1 : structured deposit balance(구조화 예금 잔액)
- X2 : time deposit balance (정기 예금 잔액)
- X3 : demand deposit balance (입출금 통장 잔액)
- X4 : financial products balance (금융 상품 잔액)
- X5 : fund balance (펀드 잔액)
- X6 : asset management balance (자산 관리 잔액)
- X7 : loan balance (대출 잔액)
- X8 : large deposit certificate balance (거액 예금 증서 잔액)

#### customers' behaviors in month Y.  
  
Column B6 and B7 only have data if month == [3, 6, 9, 12].

- B1 : mobile banking login times (로그인 횟수)
- B2 : transfer-in times (입금 횟수)
- B3 : transfer-in money amount (입금 금액)
- B4 : transfer-out times (출금 횟수)
- B5 : transfer-out money amount (출금 금액)
- **B6(6.11%) : latest transfer time (마지막 거래 시간)**
    - min값으로 대체?
- B7 : number of transfers in a season (거래 횟수?)

#### customers' important behaviors in the season Z.

- E1 : account opening date 
- **E2(4.38%) : online banking opening date** 
    - max값으로 대체?
- **E3(4.38%) : mobile banking opening date** 
    - max값으로 대체?
- ~~E4 : first online banking login date~~ 
- **E5(37.94%) : first mobile banking login date**
    - max값으로 대체?
- **E6(5.19%) : first demand deposit date**
    - 평균값으로 대체?
- ~~E7 : first time deposit date (최초 입금 일)~~
- ~~E8 : first loan date (최초 대출 일)~~
- ~~E9 : first overdue date (최초 연체 날짜)~~
- **E10(0.56%) : first cash transaction date (최초 현금 거래일)** 
    - 평균값으로 대체?
- ~~E11 : first bank-securities transfer date (최초 은행 - 증권 이체 날짜)~~
- ~~E12 : first transfer at counter date (최초 카운터에서 이체 날짜)~~
- ~~E13 : first transfer via online banking date (온라인 뱅킹을 이용한 최초 이체 날짜)~~
- ~~E14 : first transfer via mobile banking date (모바일 뱅킹을 이용한 최초 이체 날짜)~~
- E15 : maximum amount transferred out of another bank (다른 은행에서 이체된 최대 금액)
- **E16(47.17%) : maximum amount transferred out of another bank date (다른 은행에서 이체된 최대 금액이 이체된 날짜)** 
    - 평균값으로 대체?
- E17 : Maximum transfer amount from other bank (타 은행 최대 이체 금액)
- **E18(42.77%) : Maximum transfer amount from other bank date (타 은행 최대 이체 금액 이체 날짜)** 
    - 평균값으로 대체?  
    
#### customers' deposits in month Y.

- **C1(0.005%) : deposit products value (예금 상품 가치)**  
    - 0으로 대체
- **C2(0.005%) : number of deposit products (예금 상품 수)**
    - 0으로 대체  
    
#### valid customer IDs in the season Z.

####  customer information in the season Z.

- **l1(0.044%) : gender (성별)**
    - '여자'로 대체?
- l2 : age (나이)
- l3 : class
- l4 : tag
- **I5(7.99%) : occupation (직업)**
    - '무직'으로 대체?
- I6 : deposit customer tag (예금 고객 태그)
- I7 : number of products owning (보유 제품 수)
- I8 : constellation (별자리)
- ~~I9 : contribution (기부금)~~
- I10 : education level (교육 수준)
- I11 : family annual income (가구 연간 소득)
- I12 : field description
- ~~I13 : marriage description~~
- ~~I14 : occupation description~~
- I15 : QR code recipient
- I16 : VIP (vip 여부)
- I17 : online banking client
- I18 : mobile banking client
- I19 : SMS client
- I20 : WeChat Pay client